# NMR preprocessing with adaptive QC thresholds (Scanpy)

This notebook loads CellBender-filtered files per region, merges replicates 1 and 2, applies QC with histogram/KDE-based adaptive threshold suggestions (from scanpy/find_means.py), and saves preprocessed AnnData objects. Raw counts are stored in `adata.layers['counts']`.

In [ ]:
import os, sys, json
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
from pathlib import Path

# Ensure local module is importable
project_root = Path().resolve()
module_path = project_root / 'scanpy'
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path))
from find_means import (
    suggest_thresholds_for_obs,
    apply_threshold_to_obs_metric,
    interactive_select_threshold,
)

sc.settings.verbosity = 3
sc.set_figure_params(figsize=(6, 5), dpi=100)


## Input paths and region/replicate mapping
Assumes CellBender-filtered HDF5 files are available on a Windows path.

In [ ]:
# Windows path containing the filtered CellBender outputs
cellbender_dir = r'C:\Users\Antonio\Documents\Maestria\Semestre 5\Estancia\Datos transcripcion NMR\Siguiendo word\RNA NMR Single cells\cellbender\clean_data\'

files = {
    'NMR1': 'NMR1_cerebral_cortex_denoised_filtered.h5',
    'NMR2': 'NMR2_cerebral_cortex_denoised_filtered.h5',
    'NMR3': 'NMR3_hippocampus_denoised_filtered.h5',
    'NMR4': 'NMR4_hippocampus_denoised_filtered.h5',
    'NMR5': 'NMR5_midbrain_denoised_filtered.h5',
    'NMR6': 'NMR6_midbrain_denoised_filtered.h5',
}

# Define region groupings and replicates to merge
regions = {
    'cerebral_cortex': ['NMR1', 'NMR2'],
    'hippocampus': ['NMR3', 'NMR4'],
    'midbrain': ['NMR5', 'NMR6'],
}

def resolve_path(fname):
    p = os.path.join(cellbender_dir, fname)
    if not os.path.exists(p):
        raise FileNotFoundError(p)
    return p


## Helper functions

In [ ]:
def read_cellbender_h5(path):
    """
    Reads a CellBender output HDF5. Many CellBender outputs are 10x-like h5s
    compatible with sc.read_10x_h5. If this fails, adjust accordingly to the
    specific file structure.
    """
    try:
        adata = sc.read_10x_h5(path)
    except Exception as e:
        raise RuntimeError(f'Failed to read {path} as 10x h5: {e}')
    # Ensure unique variable names
    adata.var_names_make_unique()
    return adata

def annotate_basic_qc(adata: ad.AnnData, mito_prefixes=("mt-", "MT-", "Mt-")):
    # store raw counts in layer 'counts'
    if adata.layers.get('counts', None) is None:
        adata.layers['counts'] = adata.X.copy()
    # mitochondrial percentage
    mito_genes = adata.var_names.str.startswith(mito_prefixes).to_numpy()
    # Compute QC metrics
    sc.pp.calculate_qc_metrics(adata, qc_vars=[pd.Index(adata.var_names[mito_genes]).tolist()],
                               percent_top=None, log1p=False, inplace=True)
    # Standardize naming: pct_counts_mt
    if 'pct_counts_in_set1' in adata.obs:
        adata.obs.rename(columns={'pct_counts_in_set1': 'pct_counts_mt'}, inplace=True)
    # Fallback if above not created
    if 'pct_counts_mt' not in adata.obs:
        # manual compute
        if mito_genes.sum() > 0:
            mt_counts = adata[:, mito_genes].X
            if hasattr(mt_counts, 'toarray'):
                mt_counts = mt_counts.toarray()
            total = adata.X
            if hasattr(total, 'toarray'):
                total = total.toarray()
            mt_pct = 100.0 * (mt_counts.sum(axis=1).A1 if hasattr(mt_counts, 'A1') else mt_counts.sum(axis=1)) / \n                    (total.sum(axis=1).A1 if hasattr(total, 'A1') else total.sum(axis=1))
            adata.obs['pct_counts_mt'] = mt_pct
        else:
            adata.obs['pct_counts_mt'] = 0.0
    return adata

def add_region_replicate_metadata(adata: ad.AnnData, region: str, replicate: str) -> ad.AnnData:
    adata.obs['region'] = region
    adata.obs['replicate'] = replicate
    return adata

def preprocess_one(adata: ad.AnnData, region: str) -> ad.AnnData:
    # Example adaptive threshold suggestions for QC
    # total_counts: typically log1p transform and choose a floor (>=)
    out_tc = suggest_thresholds_for_obs(adata, 'total_counts', transform='log1p', show=True,
                                        save=f'scanpy/figures/{region}_total_counts_thresholds.png')
    thr_tc = interactive_select_threshold(out_tc['thresholds'])
    if thr_tc is not None:
        apply_threshold_to_obs_metric(adata, 'total_counts', thr_tc, direction='>=',
                                      key_added='qc_total_counts_ok', original_scale=out_tc['original_scale'])
    else:
        adata.obs['qc_total_counts_ok'] = True

    # n_genes_by_counts: also often log1p with a floor
    if 'n_genes_by_counts' not in adata.obs:
        # Ensure metric is available
        sc.pp.calculate_qc_metrics(adata, inplace=True)
    out_ng = suggest_thresholds_for_obs(adata, 'n_genes_by_counts', transform='log1p', show=True,
                                       save=f'scanpy/figures/{region}_n_genes_by_counts_thresholds.png')
    thr_ng = interactive_select_threshold(out_ng['thresholds'])
    if thr_ng is not None:
        apply_threshold_to_obs_metric(adata, 'n_genes_by_counts', thr_ng, direction='>=',
                                      key_added='qc_ngenes_ok', original_scale=out_ng['original_scale'])
    else:
        adata.obs['qc_ngenes_ok'] = True

    # pct_counts_mt: typically cap high-mito cells (<=) in percent units
    out_mt = suggest_thresholds_for_obs(adata, 'pct_counts_mt', transform='none', show=True,
                                      save=f'scanpy/figures/{region}_pct_counts_mt_thresholds.png')
    thr_mt = interactive_select_threshold(out_mt['thresholds'])
    if thr_mt is not None:
        apply_threshold_to_obs_metric(adata, 'pct_counts_mt', thr_mt, direction='<=',
                                      key_added='qc_mito_ok', original_scale=out_mt['original_scale'])
    else:
        adata.obs['qc_mito_ok'] = True

    # Combine QC flags
    qc_pass = adata.obs[['qc_total_counts_ok', 'qc_ngenes_ok', 'qc_mito_ok']].all(axis=1)
    adata.obs['qc_pass'] = qc_pass

    # Filter to passing cells (optional: keep a copy of unfiltered as raw)
    adata = adata[qc_pass].copy()

    # Normalize, log1p, HVGs, scaling, PCA, neighbors, UMAP
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=3000)
    adata = adata[:, adata.var['highly_variable']].copy()
    sc.pp.scale(adata, max_value=10)
    sc.tl.pca(adata, svd_solver='arpack')
    sc.pp.neighbors(adata, n_neighbors=15, n_pcs=50)
    sc.tl.umap(adata)

    return adata


## Load, merge replicates per region, annotate, preprocess, and save

In [ ]:
preprocessed = {}
output_dir = Path('scanpy') / 'processed'
output_dir.mkdir(parents=True, exist_ok=True)

for region, reps in regions.items():
    adatas = []
    for rep in reps:
        fp = resolve_path(files[rep])
        print(f'Reading {rep} from: {fp}')
        adata = read_cellbender_h5(fp)
        # Keep original counts in a layer for reference
        adata.layers['counts'] = adata.X.copy()
        # annotate region/replicate
        adata = add_region_replicate_metadata(adata, region=region, replicate=rep)
        # basic qc metrics (total_counts, n_genes_by_counts, pct_counts_mt)
        annotate_basic_qc(adata)
        adatas.append(adata)
    # Concatenate replicates per region
    adata_region = ad.concat(adatas, join='outer', label='batch', keys=reps, fill_value=0)
    # Preprocess with adaptive QC suggestions
    adata_proc = preprocess_one(adata_region, region=region)
    preprocessed[region] = adata_proc
    out_path = output_dir / f'{region}_preprocessed.h5ad'
    print(f'Saving {region} to {out_path}')
    adata_proc.write(out_path)

# Optionally, also concatenate all regions into a single object and save
all_adata = ad.concat(list(preprocessed.values()), join='outer', label='region', keys=list(preprocessed.keys()), fill_value=0)
all_out = output_dir / 'all_regions_preprocessed.h5ad'
print(f'Saving all regions to {all_out}')
all_adata.write(all_out)
